Install Require Libraries

In [8]:
!pip install pandas numpy requests

  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/f9/9b/335f9764261e915ed497fcdeb11df5dfd6f7bf257d4a6a2a686d80da4d54/requests-2.32.3-py3-none-any.whl.metadata
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/40/26/f35951c45070edc957ba40a5b1db3cf60a9dbb1b350c2d5bef03e01e61de/charset_normalizer-3.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached charset_normalizer-3.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Obtaining dependency information for idna<4,>=2.5 from https://files.pythonhosted.org/packages/e5/3e/741d8c82801c347547f8a2a06aa57dbb1992be9e948df2ea0eda2c8b79e8/idna-3.7-py3-none-any.whl.metadata
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/ca/1c/89ffc63a9605b583d5df2be791a27bc1a42b7c32bab68d

Import Libraries

In [2]:
import pandas as pd
import numpy as np

# Show all columns in the dataframe
pd.set_option('display.max_columns', None)

Extract Files from the Zips

In [2]:
import os
import zipfile
import shutil

def extract_all_zips(src_folder, dest_folder):
    # Crear la carpeta de destino si no existe
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    # Función recursiva para descomprimir archivos zip
    def extract_zip(zip_path, current_folder):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(current_folder)

            # Procesar cada archivo descomprimido
            for file_name in zip_ref.namelist():
                file_path = os.path.join(current_folder, file_name)
                if zipfile.is_zipfile(file_path):
                    # Si el archivo es un zip, descomprimirlo
                    extract_zip(file_path, current_folder)
                else:
                    # Si es otro archivo, moverlo a la carpeta de destino
                    shutil.move(file_path, os.path.join(dest_folder, file_name))

    # Recorrer la carpeta fuente
    for root, dirs, files in os.walk(src_folder):
        for file in files:
            file_path = os.path.join(root, file)
            if zipfile.is_zipfile(file_path):
                extract_zip(file_path, root)

# Carpeta origen con los archivos zip
source_folder = '/home/andromedalactea/freelance/PipelineDataLeads/data/compressed'

# Carpeta destino donde se guardarán los archivos descomprimidos
destination_folder = '/home/andromedalactea/freelance/PipelineDataLeads/data/descompressed'

# Ejecutar el script
extract_all_zips(source_folder, destination_folder)


Read CSV File

In [3]:
# Read File
leads_data = pd.read_csv('../data/Michigan.csv', on_bad_lines='skip')



/tmp/ipykernel_9851/1174662041.py:2: DtypeWarning: Columns (12,22,53,56,58) have mixed types. Specify dtype option on import or set low_memory=False.
  leads_data = pd.read_csv('../data/Michigan.csv', on_bad_lines='skip')


In [4]:
# Charge first 100 records of a template reference
template = pd.read_csv('../data/LIST_100_20240723-161841.csv', nrows=100)

In [5]:
template.columns

Index(['lead_id', 'entry_date', 'modify_date', 'status', 'status_name',
       'status_category_id', 'status_category_name', 'user',
       'vendor_lead_code', 'source_id', 'list_id', 'gmt_offset_now',
       'called_since_last_reset', 'phone_code', 'phone_number', 'title',
       'first_name', 'middle_initial', 'last_name', 'address1', 'address2',
       'address3', 'city', 'state', 'province', 'postal_code', 'country_code',
       'gender', 'date_of_birth', 'alt_phone', 'email', 'security_phrase',
       'comments', 'called_count', 'last_local_call_time', 'rank', 'owner',
       'entry_list_id'],
      dtype='object')

In [6]:
template.sample(2)

,lead_id,entry_date,modify_date,status,status_name,status_category_id,status_category_name,user,vendor_lead_code,source_id,list_id,gmt_offset_now,called_since_last_reset,phone_code,phone_number,title,first_name,middle_initial,last_name,address1,address2,address3,city,state,province,postal_code,country_code,gender,date_of_birth,alt_phone,email,security_phrase,comments,called_count,last_local_call_time,rank,owner,entry_list_id
13,1256807,2024-05-28 22:59:59,0000-00-00 00:00:00,NaN,No Answer AutoDial,UNDEFINED,Default Category -,NaN,NaN,NaN,100,-6.0,N,1,3039346337,PRES,MR COLBY BULGER,NaN,MILE HIGH DISPENSARY LLC,1350 S SHERIDAN BLVD,NaN,NaN,JEFFERSON,CO,NaN,80232,NaN,NaN,0000-00-00,NaN,NaN,NaN,NaN,6,2008-01-01 00:00:00,0,NaN,0
82,1256876,2024-05-28 22:59:59,0000-00-00 00:00:00,NaN,No Answer AutoDial,UNDEFINED,Default Category -,NaN,NaN,NaN,100,-5.0,N,1,3202357244,CHIE,MS DIANE MEINDERS,NaN,WILLMAR HEARING AID CENTER,1604 1ST ST S STE 10,NaN,NaN,KANDIYOHI,MN,NaN,56201,NaN,U,0000-00-00,NaN,NaN,NaN,NaN,6,2008-01-01 00:00:00,0,NaN,0


In [7]:
leads_data.columns

Index(['Full name', 'Industry', 'Job title', 'Sub Role', 'Industry 2',
       'Emails', 'Mobile', 'Phone numbers', 'Company Name', 'Company Industry',
       'Company Website', 'Company Size', 'Company Founded', 'Location',
       'Locality', 'Metro', 'Region', 'Skills', 'First Name', 'Middle Initial',
       'Middle Name', 'Last Name', 'Birth Year', 'Birth Date', 'Gender',
       'LinkedIn Url', 'LinkedIn Username', 'Facebook Url',
       'Facebook Username', 'Twitter Url', 'Twitter Username', 'Github Url',
       'Github Username', 'Company Linkedin Url', 'Company Facebook Url',
       'Company Twitter Url', 'Company Location Name',
       'Company Location Locality', 'Company Location Metro',
       'Company Location Region', 'Company Location Geo',
       'Company Location Street Address', 'Company Location Address Line 2',
       'Company Location Postal Code', 'Company Location Country',
       'Company Location Continent', 'Last Updated', 'Start Date',
       'Job Summary', 'Loc

In [8]:
len(leads_data)

2127118

In [9]:
# Obtener el conteo de valores nulos por columna
null_counts = leads_data.isnull().sum()

# Convertir a DataFrame
null_counts_df = null_counts.reset_index()
null_counts_df.columns = ['Column', 'Null Count']

# Exportar a CSV
null_counts_df.to_csv('null_counts.csv', index=False)

Delete the rows without a phone number

In [10]:
df_cleaned = leads_data.dropna(subset=['Mobile', 'Phone numbers'], how='all')


Format data for the final status

In [11]:
# Function to process phone numbers
def process_phone_numbers(df):
    # Initialize new columns using .loc to avoid SettingWithCopyWarning
    df.loc[:, 'phone_code'] = np.nan
    df.loc[:, 'phone_number'] = np.nan
    df.loc[:, 'alt_phone'] = np.nan
    
    # Convert columns to string to avoid attribute errors
    df['Mobile'] = df['Mobile'].astype(str)
    df['Phone numbers'] = df['Phone numbers'].astype(str)

    def extract_phone_info(phone):
        # Handle missing values
        if phone.lower() == 'nan':
            return (np.nan, np.nan)
        
        phone = phone.replace(' ', '')
        
        if phone.startswith('+'):
            phone = phone[1:]
            if len(phone) > 10 and len(phone) <= 12:
                phone_code = phone[:-10]
                phone_number = phone[-10:]
                return (phone_code, phone_number)
        return (np.nan, np.nan)

    def fill_phone_info(row):
        # Process Mobile column
        if row['Mobile'].lower() != 'nan':
            phone_code, phone_number = extract_phone_info(row['Mobile'])
            row['phone_code'] = phone_code
            row['phone_number'] = phone_number
        
        # Process Phone numbers column if Mobile is not valid
        if pd.isna(row['phone_code']) or pd.isna(row['phone_number']):
            phones = row['Phone numbers'].split(',')
            phone_code, phone_number = extract_phone_info(phones[0])
            row['phone_code'] = phone_code
            row['phone_number'] = phone_number
            
            # If there are additional numbers, add them to alt_phone
            if len(phones) > 1:
                row['alt_phone'] = ','.join(phones[1:])
        
        # If Mobile had valid data, just put Phone numbers in alt_phone
        else:
            row['alt_phone'] = row['Phone numbers']
        
        return row
    
    # Apply the function to each row of the DataFrame
    df = df.apply(fill_phone_info, axis=1)
    
    return df


# Procesar los números de teléfono
df_cleaned = process_phone_numbers(df_cleaned)

# Mostrar el DataFrame resultante
df_cleaned


/tmp/ipykernel_9851/1085458843.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'phone_code'] = np.nan
/tmp/ipykernel_9851/1085458843.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'phone_number'] = np.nan
/tmp/ipykernel_9851/1085458843.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

,Full name,Industry,Job title,Sub Role,Industry 2,Emails,Mobile,Phone numbers,Company Name,Company Industry,Company Website,Company Size,Company Founded,Location,Locality,Metro,Region,Skills,First Name,Middle Initial,Middle Name,Last Name,Birth Year,Birth Date,Gender,LinkedIn Url,LinkedIn Username,Facebook Url,Facebook Username,Twitter Url,Twitter Username,Github Url,Github Username,Company Linkedin Url,Company Facebook Url,Company Twitter Url,Company Location Name,Company Location Locality,Company Location Metro,Company Location Region,Company Location Geo,Company Location Street Address,Company Location Address Line 2,Company Location Postal Code,Company Location Country,Company Location Continent,Last Updated,Start Date,Job Summary,Location Country,Location Continent,Street Address,Address Line 2,Postal Code,Location Geo,Last Updated.1,Linkedin Connections,Inferred Salary,Years Experience,Summary,Countries,Interests,phone_code,phone_number,alt_phone
4,nicholas stauder,information technology and services,hpc systems administrator,NaN,NaN,"nstauder@gmail.com, nstauder@ix.netcom.com, ns...",+17344746913,"+17344746913, +13134341535",university of michigan,higher education,umich.edu,10001+,1817,"ann arbor, michigan, united states",ann arbor,"ann arbor, michigan",michigan,"architectural design, cad, microsoft office, p...",nicholas,j,john,stauder,1988.0,1988-07-05,male,linkedin.com/in/nicholas-stauder-a83b2822,nicholas-stauder-a83b2822,facebook.com/nstauder,nstauder,twitter.com/nstauder,nstauder,github.com/nstauder,nstauder,linkedin.com/company/university-of-michigan,NaN,NaN,"ann arbor, michigan, united states",ann arbor,"ann arbor, michigan",michigan,"42.27,-83.72",1239 kipke drive,NaN,48109,united states,north america,2020-07-01,2018-08,* High Performance Computing (HPC) systems adm...,united states,north america,300 briarcrest drive,unit 147,48104,"42.27,-83.72",2020-09-01,86,"150,000-250,000",10,Experienced systems administrator with a demon...,united states,"football, children, computers, design, technol...",1,7344746913,"+17344746913, +13134341535"
5,nicholas stauder,information technology and services,hpc systems administrator,NaN,NaN,"nstauder@gmail.com, nstauder@ix.netcom.com, ns...",+17344746913,"+17344746913, +13134341535",university of michigan,higher education,umich.edu,10001+,1817,"ann arbor, michigan, united states",ann arbor,"ann arbor, michigan",michigan,"architectural design, cad, microsoft office, p...",nicholas,j,john,stauder,1988.0,1988-07-05,male,linkedin.com/in/nicholas-stauder-a83b2822,nicholas-stauder-a83b2822,facebook.com/nstauder,nstauder,twitter.com/nstauder,nstauder,github.com/nstauder,nstauder,linkedin.com/company/university-of-michigan,NaN,NaN,"ann arbor, michigan, united states",ann arbor,"ann arbor, michigan",michigan,"42.27,-83.72",1239 kipke drive,NaN,48109,united states,north america,2020-07-01,2018-08,* High Performance Computing (HPC) systems adm...,united states,north america,300 briarcrest drive,unit 147,48104,"42.27,-83.72",2020-09-01,86,"150,000-250,000",10,Experienced systems administrator with a demon...,united states,"football, children, computers, design, technol...",1,7344746913,"+17344746913, +13134341535"
20,judy stull,retail,sales,accounts,sales,"hjstull49@gmail.com, riley.stull@gmail.com",+17022038753,"+17022038753, +17028385224, +18053790755, +161...",bath & body works,retail,bathandbodyworks.com,10001+,1990,"kalamazoo, michigan, united states",kalamazoo,"kalamazoo, michigan",michigan,NaN,judy,r,rae,stull,1949.0,NaN,female,linkedin.com/in/judy-stull-208b4644,judy-stull-208b4644,facebook.com/judy.stull,judy.stull,NaN,NaN,NaN,NaN,linkedin.com/company/bath-&-body-works,facebook.com/bathandbodyworks,twitter.com/luvbbw,"columbus, ohio, united states",columbus,"columbus, ohio",ohio,"39.96,-82.99",7 limited parkway,NaN,43068,united states,north america,2020-12-01,NaN,NaN,united states,north america,6209 poppy street,NaN,49009,"42.29,-85.58",2020-10-01,26,"70,000-85,000",3,"sales at Wil

Rename the name of some columns

In [12]:
df_cleaned.sample(2)

,Full name,Industry,Job title,Sub Role,Industry 2,Emails,Mobile,Phone numbers,Company Name,Company Industry,Company Website,Company Size,Company Founded,Location,Locality,Metro,Region,Skills,First Name,Middle Initial,Middle Name,Last Name,Birth Year,Birth Date,Gender,LinkedIn Url,LinkedIn Username,Facebook Url,Facebook Username,Twitter Url,Twitter Username,Github Url,Github Username,Company Linkedin Url,Company Facebook Url,Company Twitter Url,Company Location Name,Company Location Locality,Company Location Metro,Company Location Region,Company Location Geo,Company Location Street Address,Company Location Address Line 2,Company Location Postal Code,Company Location Country,Company Location Continent,Last Updated,Start Date,Job Summary,Location Country,Location Continent,Street Address,Address Line 2,Postal Code,Location Geo,Last Updated.1,Linkedin Connections,Inferred Salary,Years Experience,Summary,Countries,Interests,phone_code,phone_number,alt_phone
584535,lynda ciccarelli,consumer services,NaN,NaN,NaN,lyndacicc@hotmail.com,nan,"+15863294463, +15869923348, +17166025174, +158...",NaN,NaN,NaN,NaN,NaN,"macomb, michigan, united states",macomb,NaN,michigan,NaN,lynda,j,june,ciccarelli,1959.0,1959,female,linkedin.com/in/lynda-ciccarelli-96210a24,lynda-ciccarelli-96210a24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,united states,north america,45455 shoal drive,NaN,48044,"42.70,-82.95",2020-07-01,5,NaN,NaN,Owner De Jolie Salon and Spa,united states,NaN,1,5863294463,"+15869923348, +17166025174, +15866770711"
1356687,nicole christianson,insurance,account service representative and retention s...,NaN,NaN,nicolemc1978@msn.com,nan,"+15175468705, +15174496032",farmers insurance,insurance,farmers.com,10001+,1928,"fowlerville, michigan, united states",fowlerville,"detroit, michigan",michigan,"auditing, administrative assistance, leasing, ...",nicole,m,NaN,christianson,1978.0,1978-06,female,linkedin.com/in/nicole-christianson-42547111,nicole-christianson-42547111,NaN,NaN,NaN,NaN,NaN,NaN,linkedin.com/company/farmers-insurance,facebook.com/farmersinsurance,twitter.com/wearefarmers,"woodland hills, california, united states",woodland hills,"los angeles, california",california,"34.16,-118.60",6301 owensmouth avenue,NaN,91367,united states,north america,2020-09-01,2018-01,NaN,united states,north america,375 north dieterle road,NaN,48836,"42.66,-84.07",2020-07-01,189,"70,000-85,000",7,Insurance Producer at Allstate Insurance Agenc...,united states,"exercise, home improvement, reading, shooting,...",1,5175468705,+15174496032


Procesing the name into columns

In [13]:
# Columns to rename and their new names
columns_to_rename = {
    "Full name": "first_name",
    "Company Name": "last_name",
    "Street Address" : "address1",
    "Address Line 2" : "address2",
    "Location" : "city",
    "Postal Code": "postal_code",
    "Birth Date": "date_of_birth",
    "Emails" : "email",
    "Job title" : "title",
    "Gender" : "gender",
    "Middle Initial" : "middle_initial"
}

df_cleaned.rename(columns=columns_to_rename, inplace=True)

# Add a column for the state
df_cleaned['state'] = 'MI'

Format City

In [14]:
# Función para limpiar la columna 'city'
def clean_city_column(city):
    """
    Limpia la cadena de la columna 'city' eliminando las palabras 'michigan', 'united states' y comas.
    
    Args:
        city (str): La cadena de la ciudad a limpiar.
    
    Returns:
        str: La cadena de la ciudad limpiada.
    """
    if pd.isna(city):
        return city
    city = city.lower()
    words_to_remove = ['michigan', 'united states', ',']
    for word in words_to_remove:
        city = city.replace(word, '')
    return city.strip()

# Aplicar la función a la columna 'city' del DataFrame
df_cleaned['city'] = df_cleaned['city'].apply(clean_city_column)

Upper case for all values

In [15]:
# Función para convertir caracteres no numéricos a mayúsculas
def convert_non_numeric_to_upper(s):
    if pd.isna(s):
        return s
    return ''.join([char.upper() if not char.isdigit() else char for char in s])

# Aplicar la función a cada columna del DataFrame
for col in df_cleaned.columns:
    if col != 'email':
        df_cleaned[col] = df_cleaned[col].apply(lambda x: convert_non_numeric_to_upper(str(x)))

In [16]:
df_cleaned.sample(5)

,first_name,Industry,title,Sub Role,Industry 2,email,Mobile,Phone numbers,last_name,Company Industry,Company Website,Company Size,Company Founded,city,Locality,Metro,Region,Skills,First Name,middle_initial,Middle Name,Last Name,Birth Year,date_of_birth,gender,LinkedIn Url,LinkedIn Username,Facebook Url,Facebook Username,Twitter Url,Twitter Username,Github Url,Github Username,Company Linkedin Url,Company Facebook Url,Company Twitter Url,Company Location Name,Company Location Locality,Company Location Metro,Company Location Region,Company Location Geo,Company Location Street Address,Company Location Address Line 2,Company Location Postal Code,Company Location Country,Company Location Continent,Last Updated,Start Date,Job Summary,Location Country,Location Continent,address1,address2,postal_code,Location Geo,Last Updated.1,Linkedin Connections,Inferred Salary,Years Experience,Summary,Countries,Interests,phone_code,phone_number,alt_phone,state
937803,LORI JOHNSON,MARKETING AND ADVERTISING,GLOBAL RISK ADVISORY MARKETING STRATEGIST,NAN,MARKETING,"ljohnson@deloitte.com, lojohnson@deloitte.com",NAN,+15137234170,DELOITTE,MANAGEMENT CONSULTING,DELOITTE.COM,10001+,1900,GRAND RAPIDS,GRAND RAPIDS,"GRAND RAPIDS, MICHIGAN",MICHIGAN,"STRATEGY, KNOWLEDGE MANAGEMENT, ADVERTISING, M...",LORI,A,NAN,JOHNSON,NAN,NAN,FEMALE,LINKEDIN.COM/IN/LORI-JOHNSON-A6B2291,LORI-JOHNSON-A6B2291,FACEBOOK.COM/LJOHNSON1024,LJOHNSON1024,TWITTER.COM/LOJO1024,LOJO1024,NAN,NAN,LINKEDIN.COM/COMPANY/DELOITTE,FACEBOOK.COM/DELOITTE,TWITTER.COM/DELOITTE,"NEW YORK, NEW YORK, UNITED STATES",NEW YORK,"NEW YORK, NEW YORK",NEW YORK,"40.71,-74.00",30 ROCKEFELLER PLAZA,NAN,10112,UNITED STATES,NORTH AMERICA,2020-11-01,2019-12,NAN,UNITED STATES,NORTH AMERICA,NAN,NAN,NAN,"42.96,-85.66",2020-09-01,1075.0,">250,000",27.0,INTEGRATED MARKETING LEADER WITH 20 YEARS OF H...,INDIA; UNITED STATES,"CHILDREN, CIVIL RIGHTS AND SOCIAL ACTION, ENVI...",1,5137234170,NAN,MI
1781976,BROCK HOFFMAN,FINANCIAL SERVICES,REGIONAL MANAGER,NAN,NAN,NaN,+15172629413,+15172629413,PRIMERICA,FINANCIAL SERVICES,PRIMERICA.COM,1001-5000,1977,GRAND RAPIDS,GRAND RAPIDS,"GRAND RAPIDS, MICHIGAN",MICHIGAN,LEADERSHIP,BROCK,NAN,NAN,HOFFMAN,NAN,NAN,MALE,LINKEDIN.COM/IN/BROCK-HOFFMAN-28A035152,BROCK-HOFFMAN-28A035152,FACEBOOK.COM/BROCK.HOFFMAN.946,BROCK.HOFFMAN.946,NAN,NAN,NAN,NAN,LINKEDIN.COM/COMPANY/PRIMERICA,FACEBOOK.COM/PRIMERICA,TWITTER.COM/PRIMERICA,"DULUTH, GEORGIA, UNITED STATES",DULUTH,"ATLANTA, GEORGIA",GEORGIA,"34.00,-84.14",1 PRIMERICA PARKWAY,NAN,30099,UNITED STATES,NORTH AMERICA,2020-11-01,2020-06,NAN,UNITED STATES,NORTH AMERICA,NAN,NAN,NAN,"42.96,-85.66",2020-10-01,1202.0,"100,000-150,000",2.0,NAN,UNITED STATES,NAN,1,5172629413,+15172629413,MI
1491036,ANDY PETERSON,AUTOMOTIVE,GLOBAL LEAD PROGRAM ENGINEERING MANAGER,NAN,ENGINEERING,"andy1peterson@aol.com, andy.peterson@gm.com",+12313608868,"+12313608868, +13135670388",GENERAL MOTORS,AUTOMOTIVE,GM.COM,10001+,NAN,GRAND BLANC,GRAND BLANC,"FLINT, MICHIGAN",MICHIGAN,"AUTOMOTIVE, PROGRAM MANAGEMENT, MANUFACTURING,...",ANDY,N,NAN,PETERSON,NAN,NAN,MALE,LINKEDIN.COM/IN/ANDY-PETERSON-02308010,ANDY-PETERSON-02308010,FACEBOOK.COM/ANDREW.N.PETERSON.7,ANDREW.N.PETERSON.7,NAN,NAN,NAN,NAN,LINKEDIN.COM/COMPANY/GENERAL-MOTORS,FACEBOOK.COM/GENERALMOTORS,TWITTER.COM/GM,"DETROIT, MICHIGAN, UNITED STATES",DETROIT,"DETROIT, MICHIGAN",MICHIGAN,"42.33,-83.04",100 RENAISSANCE CENTER,NAN,48243,UNITED STATES,NORTH AMERICA,2020-11-01,2016-01,* IN THIS ROLE I WORK WITH ALL FUNCTIONS OF TH...,UNITED STATES,NORTH AMERICA,NAN,NAN,NAN,"42.92,-83.63",2020-11-01,615.0,">250,000",22.0,NAN,UNITED STATES,"POWER BOATING, SNOW SKIING, VACATIONS DESTINAT...",1,2313608868,"+12313608868, +13135670388",MI
1471608,JIMMY ALLEN,RESTAURANTS,OWNER,NAN,NAN,jamespallen@comcast.net,+12488822935,+12488822935,HOSPITALITY ENTERPRISES,HOSPITALITY,BIGEASY.COM,501-1000,NAN,DETROIT,DETROIT,"DETROIT, MICHIGAN",MICHIGAN,"TEAM BUILDING, ENTREPRENEURSHIP, SOCIAL MEDIA,...",JIMMY,NAN,NAN,ALLEN,NAN,NAN,MALE,LINKEDIN.COM/IN/JIM

Delete extra columns

In [17]:
df_cleaned.columns

Index(['first_name', 'Industry', 'title', 'Sub Role', 'Industry 2', 'email',
       'Mobile', 'Phone numbers', 'last_name', 'Company Industry',
       'Company Website', 'Company Size', 'Company Founded', 'city',
       'Locality', 'Metro', 'Region', 'Skills', 'First Name', 'middle_initial',
       'Middle Name', 'Last Name', 'Birth Year', 'date_of_birth', 'gender',
       'LinkedIn Url', 'LinkedIn Username', 'Facebook Url',
       'Facebook Username', 'Twitter Url', 'Twitter Username', 'Github Url',
       'Github Username', 'Company Linkedin Url', 'Company Facebook Url',
       'Company Twitter Url', 'Company Location Name',
       'Company Location Locality', 'Company Location Metro',
       'Company Location Region', 'Company Location Geo',
       'Company Location Street Address', 'Company Location Address Line 2',
       'Company Location Postal Code', 'Company Location Country',
       'Company Location Continent', 'Last Updated', 'Start Date',
       'Job Summary', 'Location Co

In [18]:
# Seleccionar solo las columnas deseadas
columns_to_keep = ['first_name', 'middle_initial', 'last_name', 'title', 'email', 'gender',
                    'phone_code', 'phone_number', 'alt_phone', 'address1', 'address2',
                    'city', 'state', 'postal_code', 'date_of_birth', ]
df_cleaned = df_cleaned[columns_to_keep]

In [19]:
df_cleaned.sample(5)

,first_name,middle_initial,last_name,title,email,gender,phone_code,phone_number,alt_phone,address1,address2,city,state,postal_code,date_of_birth
1115515,AUGUST BAECKER,NAN,BUTCHER & BAECKER CONSTRUCTION,MARKETING AND BUSINESS DEVELOPMENT MANAGER,"august.baecker@com.com, abaecker@bbconstruct.com",NAN,1,2313474722,NAN,NAN,NAN,DETROIT,MI,NAN,NAN
204232,KAREN WOLTJER,NAN,SPECTRUM HEALTH,PRACTICE MANAGER,karen.woltjer@gmail.com,FEMALE,1,6168227226,+16164034288,1265 BENT GRASS COURT,NAN,ZEELAND,MI,49464,1962-02
545203,JOSEF RIEDL,NAN,NAN,NAN,josef.riedl@meijer.com,MALE,1,6167844822,NAN,NAN,NAN,GRAND RAPIDS,MI,NAN,NAN
1371546,MEL HEATH,NAN,DSSI LLC,PURCHASING ANALYST - EUROPE,"mellymel.heath@gmail.com, mheath@directsourcin...",NAN,1,5023264300,NAN,NAN,NAN,DETROIT,MI,NAN,NAN
1654885,JEREMY BROCK,NAN,DELTA WHEEL TRUING SOLUTIONS,PRODUCT DESIGNER,NaN,MALE,1,9062806604,NAN,804 DAKOTA AVENUE,NAN,GLADSTONE,MI,49837,NAN


Generate the column for lead id

In [20]:
# Generate lead_id starting from 1000
df_cleaned['lead_id'] = range(1000, 1000 + len(df_cleaned))

/tmp/ipykernel_9851/2635797748.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['lead_id'] = range(1000, 1000 + len(df_cleaned))


Export data

In [21]:
# Número de registros por archivo
chunk_size = 90000

# Número total de registros en el DataFrame
total_records = len(df_cleaned)

# Calcula el número de archivos necesarios
num_files = (total_records // chunk_size) + (1 if total_records % chunk_size != 0 else 0)

for i in range(num_files):
    # Determina el inicio y fin de los índices para este fragmento
    start_idx = i * chunk_size
    end_idx = min(start_idx + chunk_size, total_records)
    
    # Extrae el fragmento
    df_chunk = df_cleaned.iloc[start_idx:end_idx]
    
    # Exporta el fragmento a un archivo CSV
    df_chunk.to_csv(f'michigan_{i+1}.csv', index=False)

print(f"Exportación completada en {num_files} archivos.")


Exportación completada en 4 archivos.


In [23]:
(112/45965 + 241/89760 + 244/89757 + 253/89748)/4 * 100

0.2664757358636738

In [24]:
45965 + 89760 + 89757 + 89748

315230

Filter Goverment institute  from the csv files

In [7]:
import pandas as pd
import os

# Lista de palabras clave
keywords = [
    "government", "federal", "state", "department", "agency", "ministry", 
    "commission", "authority", "administration", "court", 
    "senate", "congress", "assembly", "tribunal", "police", 
    "fire department", "military", "defense", "homeland", "security", 
    "justice", "treasury", "fiscal", "customs", "armed forces", 
    "state", "executive", "legislative", "judicial",
    "school", "universit", "college", "institute", "academy", "public school", 
    "high school", "elementary school", "middle school", "community college", 
    "education board", "school district", "national", "bank", "church", "hospital"
]
# Lista de palabras clave para filtrar por 'email'
keywords_email = [".edu", ".gov", ".us", ".mil", ".org"]

# Nombres de los archivos CSV
filenames = ["michigan_1.csv", "michigan_2.csv", "michigan_3.csv", "michigan_4.csv"]

# Procesar cada archivo
for filename in filenames:
    # Leer el archivo CSV
    df = pd.read_csv(filename)
    
    # Guardar una copia de la columna 'last_name' original
    original_last_name = df['last_name'].copy()
    
    # Convertir la columna 'last_name' a minúsculas para el filtrado
    df['last_name'] = df['last_name'].str.lower()
    
    # Filtrar las filas que NO contengan ninguna de las palabras clave en 'last_name'
    for keyword in keywords:
        df = df[~df['last_name'].str.contains(keyword)]
    
    # Reemplazar NaN en la columna 'email' por una cadena vacía
    df['email'] = df['email'].fillna('')

    # Filtrar las filas cuyo 'email' contiene dominios específicos
    for keyword in keywords_email:
        df = df[~df['email'].str.contains(keyword, case=False)]
    
    # Restaurar la columna 'last_name' original antes de guardar
    df['last_name'] = original_last_name.loc[df.index]
    
    # Guardar el DataFrame filtrado en un nuevo archivo CSV
    new_filename = os.path.splitext(filename)[0] + "_filtered.csv"
    df.to_csv(new_filename, index=False)

    print(f"Archivo procesado y guardado como: {new_filename}")


Archivo procesado y guardado como: michigan_1_filtered.csv
Archivo procesado y guardado como: michigan_2_filtered.csv
Archivo procesado y guardado como: michigan_3_filtered.csv
Archivo procesado y guardado como: michigan_4_filtered.csv


In [13]:
import requests
from requests.auth import HTTPBasicAuth

# Información proporcionada
url = "https://s174-235.apntelecom.com/main/adminapi.php"

# Información de autenticación y parámetros
user = "9546008001"
password = "@cG6Ep3X&iue$YyI"
source = "python_script"
lead_id = "1492833"
delete_lead = "Y"

# Parámetros para la solicitud
params = {
    "function": "update_lead",
    "lead_id": lead_id,
    "delete_lead": delete_lead,
    "source": source,
    "user": user,
    "pass": password
}

# Hacer la solicitud POST
response = requests.post(url, data=params)

# Verificar el estado de la respuesta
if response.status_code == 200:
    print("Lead eliminado exitosamente.")
    print(response.text)
else:
    print(f"Error al eliminar el lead: {response.status_code} - {response.text}")


Lead eliminado exitosamente.
ERROR: update_lead NO MATCHES FOUND IN THE SYSTEM: |9546008001|1492833||



In [25]:
import pandas as pd
import os
import requests
from requests.auth import HTTPBasicAuth
from concurrent.futures import ThreadPoolExecutor
from IPython.display import display, clear_output

# Lista de palabras clave para 'last_name'
keywords = [
    "government", "federal", "state", "department", "agency", "ministry", 
    "commission", "authority", "administration", "court", 
    "senate", "congress", "assembly", "tribunal", "police", 
    "fire department", "military", "defense", "homeland", "security", 
    "justice", "treasury", "fiscal", "customs", "armed forces", 
    "state", "executive", "legislative", "judicial",
    "school", "university", "college", "institute", "academy", "public school", 
    "high school", "elementary school", "middle school", "community college", 
    "education board", "school district", "national", "bank", "church", "hospital"
]

# Lista de palabras clave para filtrar por 'email'
keywords_email = [".edu", ".gov", ".us", ".mil", ".org"]

# Información de autenticación y parámetros generales para la API
url = "https://s174-235.apntelecom.com/main/adminapi.php"
user = "9546008001"
password = "@cG6Ep3X&iue$YyI"
source = "python_script"

# Nombres de los archivos CSV
filenames = ["../data/uploaded_michigan_1.csv", "../data/uploaded_michigan_1.csv", "../data/uploaded_michigan_3.csv", "../data/uploaded_michigan_4.csv"]

# Variable global para el contador de eliminaciones
global_deleted_count = 0

# Función para eliminar un lead
def delete_lead(lead_id):
    global global_deleted_count

    params = {
        "function": "update_lead",
        "lead_id": lead_id,
        "delete_lead": "Y",
        "source": source,
        "user": user,
        "pass": password
    }

    response = requests.post(url, data=params)

    if response.status_code == 200:
        global_deleted_count += 1

        # Mostrar el conteo de registros eliminados en tiempo real
        clear_output(wait=True)
        display(f"Registros eliminados hasta ahora: {global_deleted_count}")
    else:
        return {
            "lead_id": lead_id,
            "status_code": response.status_code,
            "error_message": response.text
        }

# Función para procesar un archivo
def process_file(filename):
    # Leer el archivo CSV
    df = pd.read_csv(filename)
    
    # Guardar una copia de la columna 'last_name' original
    original_last_name = df['last_name'].copy()
    
    # Convertir la columna 'last_name' a minúsculas para el filtrado
    df['last_name'] = df['last_name'].str.lower()
    
    # Filtrar las filas que SI contengan alguna de las palabras clave en 'last_name'
    filtered_df = df[df['last_name'].str.contains('|'.join(keywords), case=False, na=False)]
    
    # Reemplazar NaN en la columna 'email' por una cadena vacía
    filtered_df['email'] = filtered_df['email'].fillna('')
    
    # Filtrar las filas cuyo 'email' contiene dominios específicos
    filtered_df = filtered_df[filtered_df['email'].str.contains('|'.join(keywords_email), case=False, na=False)]
    
    # Restaurar la columna 'last_name' original antes de guardar
    filtered_df['last_name'] = original_last_name.loc[filtered_df.index]
    
    # Variables para almacenar los resultados locales
    errors = []

    # Crear un ThreadPoolExecutor para eliminar los leads en paralelo
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(delete_lead, lead_id): lead_id for lead_id in filtered_df['lead_id']}
        
        for future in futures:
            result = future.result()
            if result:
                errors.append(result)
    
    print(f"Archivo procesado: {filename}")
    print(f"Errores encontrados en {len(errors)} registros.") if errors else None
    
    return len(filtered_df), errors

# Procesar los archivos en paralelo
with ThreadPoolExecutor(max_workers=len(filenames)) as executor:
    results = executor.map(process_file, filenames)

# Resumen de los resultados
total_deleted = 0
total_errors = []

for deleted_count, errors in results:
    total_deleted += deleted_count
    total_errors.extend(errors)

print(f"Total de registros eliminados: {total_deleted}")
print(f"Total de errores: {len(total_errors)}")
if total_errors:
    print("Errores detallados:")
    for error in total_errors:
        print(f"Lead ID: {error['lead_id']}, Error: {error['status_code']} - {error['error_message']}")


/tmp/ipykernel_1038/1360146266.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['email'] = filtered_df['email'].fillna('')


Archivo procesado: ../data/uploaded_michigan_4.csv


/tmp/ipykernel_1038/1360146266.py:67: DtypeWarning: Columns (7,27,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
/tmp/ipykernel_1038/1360146266.py:67: DtypeWarning: Columns (7,27,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
/tmp/ipykernel_1038/1360146266.py:67: DtypeWarning: Columns (7,27,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
/tmp/ipykernel_1038/1360146266.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['email'] = filtered_df['email'].fillna('')


Archivo procesado: ../data/uploaded_michigan_1.csv
Archivo procesado: ../data/uploaded_michigan_1.csv
Archivo procesado: ../data/uploaded_michigan_3.csv
Total de registros eliminados: 0
Total de errores: 0


In [26]:
print(100*310)

31000
